In [ ]:
import chess
import chess.pgn
import chess.svg
import chess.engine
from IPython.display import display, SVG


valores_pecas = {'p': 1, "P": 1, 'n': 3, "N": 3,
                 'b': 3, "B": 3, 'k': 5, "K": 5, 'q': 9, "Q": 9}

board = chess.Board()

#printa o tabuleiro
def print_chess_board(board, size=400):
    svg_data = chess.svg.board(board=board, size=size)
    display(SVG(svg_data))


#codigo base para jogar
def realizar_jogada(tabuleiro, movimento):
    if movimento in tabuleiro.legal_moves:
        tabuleiro.push(movimento)
        
        # Verificar se a jogada envolve uma promoção de peão
        if movimento.promotion is not None:
            # Solicitar ao usuário que escolha a peça para a promoção
            escolha_promocao = input("Escolha a peça para promoção (Q, R, B, N): ").upper()
            
            # Mapear a escolha do usuário para a peça correspondente
            mapa_promocao = {'Q': chess.QUEEN, 'R': chess.ROOK, 'B': chess.BISHOP, 'N': chess.KNIGHT}
            
            # Verificar se a escolha do usuário é válida
            if escolha_promocao in mapa_promocao:
                # Promover o peão para a peça escolhida
                tabuleiro.pop()
                tabuleiro.push(chess.Move(movimento.from_square, movimento.to_square, promotion=mapa_promocao[escolha_promocao]))
            else:
                print("Escolha inválida. O peão será promovido para uma rainha por padrão.")
                tabuleiro.pop()
                tabuleiro.push(chess.Move(movimento.from_square, movimento.to_square, promotion=chess.QUEEN))
        
        print(f"\nMovimento: {movimento.uci()}")
        print_chess_board(tabuleiro)
        return True
    else:
        print(f"\nMovimento inválido: {movimento.uci()}")
        return False

#função que usa o stockfish para fazer a jogada
def stockfish(tabuleiro, engine):
    # configurar do nível de dificuldade
    engine.configure({"Skill Level": 0})
    
    #configurar tempo minimo de pensamento
    engine.configure({"Move Overhead": 0.5})
    
    #habilitar o NNUE
    engine.configure({"Use NNUE": True})
    
    #configurar ram usada
    engine.configure({"Hash": 5120})
    
    # Clone do tabuleiro
    tabuleiro_clone = tabuleiro.copy()

    # avaliando da melhor jogada
    resultado = engine.play(tabuleiro_clone, chess.engine.Limit(time=2.0))
    movimento_escolhido = resultado.move

    print(f"Stockfish escolheu o movimento: {movimento_escolhido.uci()}")

    if resultado.move in tabuleiro_clone.legal_moves:
        tabuleiro.push(movimento_escolhido)

    print_chess_board(tabuleiro)

#função de escolha de cor
def escolher_cor():
    cor = input("Escolha a cor (B para brancas, P para pretas): ").upper()
    while cor not in ['B', 'P']:
        print("Escolha inválida. Tente novamente.")
        cor = input("Escolha a cor (B para brancas, P para pretas): ").upper()
    return cor

def jogar():
    cor_jogador = escolher_cor()
    
    tabuleiro = chess.Board()
    print("Tabuleiro inicial:")
    print_chess_board(tabuleiro)

    engine = chess.engine.SimpleEngine.popen_uci("C:\\Users\\xbea3\\Desktop\\chess challenge\\chess-challenge\\stockfish-windows-x86-64-avx2\\stockfish\\stockfish-windows-x86-64-avx2.exe")


    while not tabuleiro.is_game_over():
        if (cor_jogador == 'B' and tabuleiro.turn == chess.WHITE) or (cor_jogador == 'P' and tabuleiro.turn == chess.BLACK):
            # Vez das brancas ou pretas, dependendo da escolha do jogador
            user_input = input("Jogada (em formato UCI, ex: 'e2e4'): ")
            user_move = chess.Move.from_uci(user_input)

            if realizar_jogada(tabuleiro, user_move):
                    print(f"A jogada das {cor_jogador} foi: {user_input}")
            else:
                    print("Movimento inválido. Tente novamente.")
        else:
            # Vez das pretas ou brancas, dependendo da escolha do jogador
            stockfish(tabuleiro, engine)

    print("O jogo acabou.")
    if tabuleiro.is_checkmate():
        print("Xeque-mate!")
    elif tabuleiro.is_stalemate():
        print("Empate por afogamento.")
    elif tabuleiro.is_insufficient_material():
        print("Empate por material insuficiente.")
    elif tabuleiro.is_seventyfive_moves():
        print("Empate por regra dos 75 movimentos.")
    elif tabuleiro.is_fivefold_repetition():
        print("Empate por repetição de posição.")

    engine.quit()


jogar()